## Blog Post: Multi-Processing

This notebook will introduce and walk you through some aspects of the Multi-Processing library.

This particular library allows us to write parallelizable code that will speed up the execution of our programs and allow us to obtain results rapidly.

### What do I mean by parallelizable?

Let's say that you had 17 URL's that you wanted to scrape. Without using the multi-processing library, you would probably scrape each URL in a _serial_ manner. By this, I mean that you would scrape one URL at a time, waiting for the result of your scraping function to return a result, before proceding to the next URL.

Now, this can be quite time consuming, depending on the information you are trying to extract. Wouldnt it be better if you could scrape all 17 links at the same time?

This is precisely what the multi-processing library lets you do - you can see this in action by looking at one of my previous projects: https://tinyurl.com/yd4juxjz

By writing parallel code, you can utilise all the cores in your machine! A single python interpreter is using only 1 core to do all computations. This is why python code runs in a _serial_ manner. This is also related to pythons well known [GIL](https://wiki.python.org/moin/GlobalInterpreterLock)

For those of you who think this is too good to be true, I'll give you a short example that will show code running in parallel before taking a deeper dive into the library.

In [1]:
from multiprocessing import Pool, cpu_count
import time

cpu_count() # I have 8 cores on my computer - I could use all of them over time by writing parallelizable code.

8

In [2]:
data = (["Ibrahim", 5], ["Jennifer", 2], ["Juan", 1], ["Andrew", 3], ["Anna", 7])

In [3]:
def process_data(arg):
    print(f'Process {arg[0]} starting and will complete in {arg[1]} seconds')
    time.sleep(arg[1])
    print(f'Process {arg[0]} is now finished!')

In [4]:
def execute_pool():
    p = Pool(2)  # Run 2 processes at a time.
    p.map(process_data, data)
    p.terminate() # terminte all processes in the pool
    p.join() # exit the pool

In [5]:
execute_pool()

Process Ibrahim starting and will complete in 5 seconds
Process Jennifer starting and will complete in 2 seconds
Process Jennifer is now finished!
Process Juan starting and will complete in 1 seconds
Process Juan is now finished!
Process Andrew starting and will complete in 3 seconds
Process Ibrahim is now finished!
Process Anna starting and will complete in 7 seconds
Process Andrew is now finished!
Process Anna is now finished!


The above print out might look like sequential output - but it isnt! Be sure to run the above code yourself. Even play with the number of processes and see how it changes the results!

### Process Class

Let's now introduce the Process class. This class allows your to manually create processes tht you can then execute in parallel.

Most importantly, when using the process class, the function you want to run in parallel is called the target. Furthermore, you must provide the arguments of the target function.

Let's look at an example to make this clear:

In [6]:
from multiprocessing import Process

In [7]:
def say_my_name(name):
    print(f'My name is {name}')

In [8]:
names = ['Ibrahim', 'Juan', 'Andrew', 'Jennifer', 'Mary', "Elizabeth"]
to_process = []

In [9]:
for name in names:
    proc = Process(target=say_my_name, args=(name,)) # You must have the trailing comma
    to_process.append(proc)

From the above, we see that the `args` keyword argument is used to specify the input to the `say_my_name` function.

Now, if you want multiple argument to be passed to the function - it is better to slightly alter the say_my_name function to expect a tuple. You can then break out the tuple internally and run your operations.

It is very common to write functions that are specifically designed to be executed in parallel.

In [10]:
to_process

[<Process(Process-3, initial)>,
 <Process(Process-4, initial)>,
 <Process(Process-5, initial)>,
 <Process(Process-6, initial)>,
 <Process(Process-7, initial)>,
 <Process(Process-8, initial)>]

Above, we can see that we have a list of processes - the length of this list corresponding the number of elements in the `names` list!

Now, in order to execute these processes, we need to iterate over this process list, start the process. We then need to exit the completed processes.

It is important to note here, that even though we are iterating through the `to_process` list in a sequential manner via a `for` loop. The results do **not** need to be in that same order.

The order of the results will correspond to which process was completed first!

In this example, the results will most likely be in the same order as the iteration of the `for` loop as we are running the incredibly simple function `say_my_name`.

Later on, I will show you how to ensure order while exploiting parallelism.

In [11]:
# .start() executes a process

for p in to_process:
    p.start()

My name is Ibrahim
My name is Juan
My name is Andrew
My name is Jennifer
My name is Mary
My name is Elizabeth


The above code block shows that the processes were executed!

In [12]:
to_process

[<Process(Process-3, stopped)>,
 <Process(Process-4, stopped)>,
 <Process(Process-5, stopped)>,
 <Process(Process-6, stopped)>,
 <Process(Process-7, stopped)>,
 <Process(Process-8, stopped)>]

The above shows how the processes in our list are now in a stopped state. This means they have executed.

It is a best practice to formally terminate all processes so that they are no lingering in the background and eating up resources.

In [13]:
# .join() terminates and exits a process safely.

for p in to_process:
    p.join()

### Queue Class

Sometimes, we wish to share data across different processes. This can get messy with processes interfering with each other and results getting jumbled.

However, the built in Queue class will allow you to store data in a FIFO structure that can easily be passed between processes.

Let's do a simple example below where we store some generated passwords via processes and store the results in a queue.

In [14]:
import random
import string
from multiprocessing import Queue

queue = Queue()

def create_password(length):
    cipher = ''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(length))
    queue.put(cipher)

In [15]:
to_process = [Process(target=create_password, args=(i,)) for i in range(5,13)]

In [16]:
to_process

[<Process(Process-9, initial)>,
 <Process(Process-10, initial)>,
 <Process(Process-11, initial)>,
 <Process(Process-12, initial)>,
 <Process(Process-13, initial)>,
 <Process(Process-14, initial)>,
 <Process(Process-15, initial)>,
 <Process(Process-16, initial)>]

In [17]:
for p in to_process:
    p.start()

This time, we see no output! This is because we do not have a call to the `print` function in `create_password`.

In [18]:
for p in to_process:
    p.join()

Ok, so how do we get the results of our processes? They are in the queue!

In order to get the results, which are in FIFO order, we need to use the `.get()` method.

This needs to be done for **every** result in the queue. I like to use a list comprehension to extract everything.

In [19]:
output = [queue.get() for _ in to_process]

In [20]:
output

['D4OU2',
 'DB9C1H',
 '6VFZHYQ',
 '5VYFZHAI',
 'VBWVEHV5L',
 'FX520HRLHD',
 'I0J1VL9D2GW',
 'Z0BL6J6VHSQ9']

Now, we have our resultsin the order that they were **successfully** processed in. Again - this is not guarnateed to be the same order as the iteration of the `for` loop used to create the processes.

To further illustrate why the `Queue` class is useful, let me show you another example

In [21]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [22]:
def load_queue(data, q):
    
    for item in data:
        print(f'Placing {item} in queue')
        q.put(item)

In [23]:
def process_queue(data, q):
    for _ in data:
        data = q.get()
        print(f'currently processing {data}')
        processed = sigmoid(data)
        print(f'Result for {data} is {processed}')

In the code block below, notice how we will have to processes accessing the queue at the same time!

In [24]:
q = Queue()
data = np.arange(0.1, 1, 0.1)
process_1 = Process(target=load_queue, args=(data, q))
process_2 = Process(target=process_queue, args=(data, q))
process_1.start()
process_2.start()

q.close() # Indicate that no more data will be put on this queue by the current process. The background thread will quit once it has flushed all buffered data to the pipe.
q.join_thread() # ensures that all data in the buffer has been flushed to the pipe.

process_1.join()
process_2.join()

Placing 0.1 in queue
Placing 0.2 in queue
Placing 0.30000000000000004 in queue
currently processing 0.1
Placing 0.4 in queue
Result for 0.1 is 0.52497918747894
currently processing 0.2
Placing 0.5 in queue
Placing 0.6 in queue
Result for 0.2 is 0.549833997312478
currently processing 0.30000000000000004
Placing 0.7000000000000001 in queue
Placing 0.8 in queue
Result for 0.30000000000000004 is 0.574442516811659
currently processing 0.4
Placing 0.9 in queue
Result for 0.4 is 0.598687660112452
currently processing 0.5
Result for 0.5 is 0.6224593312018546
currently processing 0.6
Result for 0.6 is 0.6456563062257954
currently processing 0.7000000000000001
Result for 0.7000000000000001 is 0.6681877721681662
currently processing 0.8
Result for 0.8 is 0.6899744811276125
currently processing 0.9
Result for 0.9 is 0.7109495026250039


Awesome, from the above, we can really see the power of multi-processing. The use of the queue class allows our processes to work in harmony!

### Pool Class

The Pool class will allow you to maintain order in your results. Furthermore, the Pool class will allow you to use the following methods:

- `map` -> just like the built-in `map` function

- `apply` -> just like the built-in `apply` function

- `map_async` -> results will be returned in the order processed.

- `apply_async` -> results will be returned in the order processed/

Note that async means asynchronous.

Let's just jump into how to use these with examples!

A pool, is exactly what is sounds like. Imagine a swimming pool filled with processes. Each process in the pool will need to be executed in parallel with all other processes.

Thanks to `map_async` and `apply_async` we can mimic the beahviour of the `Process` class - that is, unordered outputs!

In [25]:
# apply example

pool = Pool(processes=3) # 3 processes to run in parallel at a time
output = [pool.apply(sigmoid, args=(i,)) for i in np.arange(0.1,1,0.1)]
output

[0.52497918747894001,
 0.54983399731247795,
 0.57444251681165903,
 0.598687660112452,
 0.62245933120185459,
 0.6456563062257954,
 0.66818777216816616,
 0.6899744811276125,
 0.71094950262500389]

In [26]:
# map example - be sure to check the beginning of notebook for another example.

output2 = pool.map(sigmoid, np.arange(0.1,1,0.1))
output2

[0.52497918747894001,
 0.54983399731247795,
 0.57444251681165903,
 0.598687660112452,
 0.62245933120185459,
 0.6456563062257954,
 0.66818777216816616,
 0.6899744811276125,
 0.71094950262500389]

In [27]:
# map async example

output3 = pool.map_async(sigmoid, np.arange(0.1,1,0.1))
output3.get() #Notice how the .get() method returns ALL results here

[0.52497918747894001,
 0.54983399731247795,
 0.57444251681165903,
 0.598687660112452,
 0.62245933120185459,
 0.6456563062257954,
 0.66818777216816616,
 0.6899744811276125,
 0.71094950262500389]

In [28]:
# apply async example

apply_objects = [pool.apply_async(sigmoid, args=(i, )) for i in np.arange(0.1,1,0.1)]
output4 = [result.get() for result in apply_objects] # Notice that .get() method applied to every element here
output4

[0.52497918747894001,
 0.54983399731247795,
 0.57444251681165903,
 0.598687660112452,
 0.62245933120185459,
 0.6456563062257954,
 0.66818777216816616,
 0.6899744811276125,
 0.71094950262500389]

Brilliant - you have now seen all the cases within the `Pool` class. The above example is simple, however, it illustrates the necessary methods - feel free to do your own benchmarking tests.

Be creative with the code setup. Remember that for multiple arguments to be passed, it is often better to modify the target function to accept a tuple or input parameters. Another way is to use a tuple of list with `Pool.map` to pass function arguments!

Also be sure to read out the amazing [documentation](https://docs.python.org/3.6/library/multiprocessing.html) for this library - I only just scrateched the surface with this notebook!

As always - feel free to contact me: igabr@uchicago.edu or [@Gabr\_Ibrahim](https://twitter.com/Gabr_Ibrahim)